<a href="https://colab.research.google.com/github/syoone/folium_example/blob/main/folium_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
print(os.getcwd())
!ls

# 실행시 등장하는 URL을 클릭하여 허용해주면 인증KEY가 나타난다. 복사하여 URL아래 빈칸에 붙여넣으면 마운트에 성공하게된다.
from google.colab import drive
drive.mount('./MyDrive')

In [ ]:
cd MyDrive/MyDrive

In [ ]:
import numpy as np # NumPy is the fundamental package for scientific computing

import pandas as pd # Pandas is an easy-to-use data structures and data analysis tools
pd.set_option('display.max_columns', None) # To display all columns

import matplotlib.pyplot as plt # Matplotlib is a python 2D plotting library
%matplotlib inline 
# A magic command that tells matplotlib to render figures as static images in the Notebook.

import seaborn as sns # Seaborn is a visualization library based on matplotlib (attractive statistical graphics).
sns.set_style('whitegrid') # One of the five seaborn themes
import warnings
warnings.filterwarnings('ignore') # To ignore some of seaborn warning msg

from scipy import stats, linalg

from matplotlib import rcParams
import scipy.stats as st

import folium # for map visualization
from folium import plugins

In [ ]:
train = pd.read_csv("./dataset/train.csv", parse_dates=['date'])
test = pd.read_csv("./dataset/test.csv", parse_dates=['date'])

In [ ]:
# Why are the following features converted to category type?
train['waterfront'] = train['waterfront'].astype('category') #,ordered=True)
train['view'] = train['view'].astype('category') #,ordered=True)
train['condition'] = train['condition'].astype('category') #,ordered=True)
train['grade'] = train['grade'].astype('category') #,ordered=False) # Why are these ordered 'False'?
train['zipcode'] = train['zipcode'].astype(str)
train.head(2) # Show the first 2 lines

In [ ]:
print(train.shape)
print(train.nunique())

In [ ]:
print(train.info())

In [ ]:
# Knowing the Price variable
f, ax = plt.subplots(1, 2, figsize = (14, 6))
sns.distplot(train['price'], ax=ax[0])
ax[0].set_title('Price Distribution')
plt.xlim()

sns.scatterplot(range(train.shape[0]), train['price'].sort_values(), ax=ax[1], marker="x")
ax[1].set_title('Price Curve Distribution', fontsize=15)

plt.show()

In [ ]:
price_des = train.describe()['price']
price_des.astype('int')

### 타겟인 가격 데이터의 값은 한쪽으로 많이 치우쳐저 있네요

## Map visualization

In [ ]:
houses_map = folium.Map(location = [train['lat'].mean(), train['long'].mean()], zoom_start = 10)
lat_long_data = train[['lat', 'long']].values.tolist()
h_cluster = folium.plugins.FastMarkerCluster(lat_long_data).add_to(houses_map)

houses_map

### 첫번째로 lat, long 데이터를 이용해서 실제 지도에 몇개씩 분포해 있는지 MarkerCluster로 표현했습니다. 이렇게 MarkerClouster로 표현하게 되면 쉽게 분포를 확인할 수 있습니다.

In [ ]:
houses_heatmap = folium.Map(location = [train['lat'].mean(), train['long'].mean()], zoom_start=9)
houses_heatmap.add_children(plugins.HeatMap([[row['lat'], row['long']] for name, row in train.iterrows()]))
houses_heatmap

#### 두번째로 heatmap으로 표현해볼 수 있습니다. heatmap은 위의 MarkerCluster와 유사한데 단지 형태를 그라데이션으로 표현한 것 뿐입니다.

#### 하지만, Heatmap과 MarkerCluster 로는 집의 분포는 쉽게 확인할 수 있으나, 각 Feature별 분포를 확인하기 위해서는 (예를 들어, 영역별로 집값의 분포, 평방미터의 분포 등) 다른 방법을 사용해야 합니다.

#### 일단 먼저 zipcode를 활용해서 지도에 각 영역을 표시한 다음에 그 zipcode에 해당하는 데이터의 feature 정보를 지도에 표현해봤습니다.

#### https://github.com/harlfoxem/House_Price_Prediction/blob/master/zipcode_king_county.geojson

#### 위 사이트에서 zipcode에 따른 영역 표시 파일(geojson)을 가져올 수 있습니다. 제가 이미 이 커널에 추가해 놓았으니 그대로 따라하시면 됩니다. geojson 파일은 위 사이트를 들어가보시면 알 수 있듯이 데이터 내부에 있는 zipcode에 따라 folium 지도에 영역을 표시해 놓은 것입니다.


In [ ]:
zipcode_data = train.groupby('zipcode').aggregate(np.mean)

In [ ]:
zipcode_data.reset_index(inplace=True)

In [ ]:
train['count'] = 1
count_house_zipcode = train.groupby('zipcode').sum()
count_house_zipcode.reset_index(inplace=True)
count_house_zipcode = count_house_zipcode[['zipcode', 'count']]
train.drop(['count'], axis = 1, inplace=True)

In [ ]:
zipcode_data = zipcode_data.join(count_house_zipcode.set_index('zipcode'), on='zipcode')

In [ ]:
zipcode_data.head()

In [ ]:
def show_zipcode_map(col, bins):
    geo_path = './dataset/zipcode_king_county.geojson'
    zipcode = folium.Map(location=[train['lat'].mean(), train['long'].mean()], zoom_start=9)
    zipcode.choropleth(
        geo_data=geo_path, 
        data=zipcode_data, 
        columns=['zipcode', col],
        key_on='feature.properties.ZCTA5CE10',
        fill_color='OrRd',
        fill_opacity=0.6, 
        line_opacity=0.2,
        bins=bins) # bins로 조절!
#     zipcode.save(col + '.html')
    return zipcode

In [ ]:
show_zipcode_map('count', 9)

#### 위 지도는 Marker 지도, HeatMap 과 같은 count map 입니다. zipcode 별로 나눠서 보니 뭔가 더 확실히 보이는거 같아요! 이제 몇몇 feature 별로 확인해봅시다.

#### feature 설명입니다. ( https://www.kaggle.com/c/2019-2nd-ml-month-with-kakr/data)

### Price

#### 먼저 각 zipcode별로 가격 분포를 확인해보겠습니다. 각 zipcode 별로 해당 feature의 평균값이 맵에 출력됩니다.

In [ ]:
show_zipcode_map('price', 7)

"""
지도 zipcode 섹션 별로, 타겟(price)의 분포를 한눈에 알 수 있습니다. 자세히 보니 Seatle 바로 오른쪽이 유난히 값이 비싼 것을 알 수 있네요!

찾아보니.. 뉴스기사

시애틀 동쪽이 유난히 부자들이 사나봐요.. 우리나라 강남같이..?


sqft_living, sqft_living15, sqft_lot, sqft_lot15

주거공간의 평방미터와 부지(땅)의 평방피트를 의미합니다.
위 feature와 sqft_lot(부지의 평방미터)와의 차이는 아마 집앞 마당의 유무와, 크기의 차이가 아닐까 생각합니다.
(매우 중요한 feature로 사용될 수 있을거 같아요!)
"""

In [ ]:
show_zipcode_map('sqft_lot', 9)

In [ ]:
show_zipcode_map('sqft_lot15', 9)

In [ ]:
show_zipcode_map('sqft_living', 9)

In [ ]:
show_zipcode_map('sqft_living15', 9)

"""
sqft_living(15), sqft_lot(15) 총 4가지 feature를 확인해보면 맵 분포는 비슷한데 스케일링의 차이가 있는거 같습니다. 마치 15feature가 반대 피쳐에 비해 다운스케일링 된거 같은 느낌이 드는데요. 이것만 가지고는 확신할 수 없으니 더 알아보도록하죠.
"""

In [ ]:
plt.figure(figsize=(12,6))
sns.distplot(np.log(train['sqft_lot']), hist=False, color='r', label='lot')
sns.distplot(np.log(train['sqft_lot15']), hist=False, color='b', label='lot15')
plt.show()

"""
sqft_lot, sqft_lot15 feature의 경우 너무 편향이 심해서 log화 한 후 그래프로 나타내봤습니다. 편향이 심하다는 것은 분포를 벗어난 아웃라이어가 많다는 이야기입니다. 뭔가 스케일링 된 증거는 찾을 수 없고 비교적 비슷한 분포를 그리고 있네요.
"""

In [ ]:
plt.figure(figsize=(12,6))
sns.distplot((train['sqft_living']), hist=False, color='r', label='living')
sns.distplot((train['sqft_living15']), hist=False, color='b', label='living15')
plt.show()

"""
sqft_living도 마찬가지네요. 1500 값이 living15가 living에 비해 50프로 정도 많이 분포해 있네요. 그리고, 빨간색 선(living)을 보니 lot feature와 마찬가지로 아웃라이어가 분포하고 있는 모양입니다.
"""

In [ ]:
plt.figure(figsize=(12,6))
sns.scatterplot(x=train.index, y='sqft_lot', data=train, color='r', label='lot')
sns.scatterplot(x=train.index, y='sqft_lot15', data=train, color='b', label='lot15')
plt.show()

In [ ]:
plt.figure(figsize=(12,6))
sns.scatterplot(x=train.index, y=train['sqft_living'], color='r', label='living')
sns.scatterplot(x=train.index, y=train['sqft_living15'], color='b', label='living15')
plt.show()

"""
스캐터 플랏을보니 lot, living 피쳐의 값의 범위가 15값에 비해 넓게 분포해 있는거 같습니다.
뭔가 15값은 밀집되어 있는 듯한 느낌인 반면에, living, lot 피쳐는 좀 튀는 느낌이랄까요?
"""

In [ ]:
round(train.describe()[['sqft_living', 'sqft_living15', 'sqft_lot', 'sqft_lot15']], 0)

"""
해당 피쳐 Describe를 살펴보니 역시 std에서 15 피쳐들이 작습니다. 마치 정리된 듯한 느낌이에요.
그리고, 이 Describe에서 가장 튀는 값이 하나 있네요. 바로 Max값잆니다.
"""

In [ ]:
round(train.describe().loc[['75%', 'max'], ['sqft_living', 'sqft_living15', 'sqft_lot', 'sqft_lot15']], 0)

#### 와.. 몇배나 큰 집인거죠..? 상당히 큰 아웃라이어가 보입니다.

In [ ]:
plt.figure(figsize=(12,6))
sns.boxplot(data=train[['sqft_living', 'sqft_living15']], orient='h')
plt.show()

In [ ]:
plt.figure(figsize=(18,6))
sns.boxplot(data=np.log(train[['sqft_lot', 'sqft_lot15']]), orient='h')
plt.show()

"""
Boxplot은 그래프를 통해 지금까지 알아봤던 분포, 표준편차, 평균, 아웃라이어를 한눈에 알아볼 수 있습니다. (lot 피쳐는 로그를 취했다는 점 주의하세요!)
이 아웃라이어를 어떻게 처리하냐에 따라 모델의 성능이 많이 달라질것 같습니다.
"""

# [Python] 지도 데이터 시각화 - Folium 기초 실습하기

In [ ]:
from pandas import DataFrame

#예제 데이터 만들기

ex = {'경도' : [127.061026,127.047883,127.899220,128.980455,127.104071,127.102490,127.088387,126.809957,127.010861,126.836078
                ,127.014217,126.886859,127.031702,126.880898,127.028726,126.897710,126.910288,127.043189,127.071184,127.076812
                ,127.045022,126.982419,126.840285,127.115873,126.885320,127.078464,127.057100,127.020945,129.068324,129.059574
                ,126.927655,127.034302,129.106330,126.980242,126.945099,129.034599,127.054649,127.019556,127.053198,127.031005
                ,127.058560,127.078519,127.056141,129.034605,126.888485,129.070117,127.057746,126.929288,127.054163,129.060972],
     '위도' : [37.493922,37.505675,37.471711,35.159774,37.500249,37.515149,37.549245,37.562013,37.552153,37.538927,37.492388
              ,37.480390,37.588485,37.504067,37.608392,37.503693,37.579029,37.580073,37.552103,37.545461,37.580196,37.562274
              ,37.535419,37.527477,37.526139,37.648247,37.512939,37.517574,35.202902,35.144776,37.499229,35.150069,35.141176
              ,37.479403,37.512569,35.123196,37.546718,37.553668,37.488742,37.493653,37.498462,37.556602,37.544180,35.111532
              ,37.508058,35.085777,37.546103,37.483899,37.489299,35.143421],
     '구분' : ['음식','음식','음식','음식','생활서비스','음식','음식','음식','음식','음식','음식','음식','음식','음식','음식'
             ,'음식','음식','소매','음식','음식','음식','음식','소매','음식','소매','음식','음식','음식','음식','음식','음식'
             ,'음식','음식','음식','음식','소매','음식','음식','의료','음식','음식','음식','소매','음식','음식','음식','음식'
             ,'음식','음식','음식']}

ex=DataFrame(ex)
ex.head()

### Folium 라이브러리를 import하고 지도 띄우기

In [ ]:
import folium

#지도의 중심을 지정하기 위해 위도와 경도의 평균 구하기
lat = ex['위도'].mean()
long = ex['경도'].mean()

#지도 띄우기
m = folium.Map([lat,long],zoom_start=9)
m

#### 이제 지도 위에 데이터들을 표시해봅니다.
Folium 버전 문제로 한글 폰트가 jupyter 결과창에 안나오는데 이걸 해결하기 위해 html 파일로 저장시켜줍니다.

#### example.html 파일을 열어주면 데이터들이 찍혀있는 것을 확인할 수 있습니다.

In [ ]:
for i in ex.index:
    sub_lat =  ex.loc[i,'위도']
    sub_long = ex.loc[i,'경도']
    
    title = ex.loc[i,'구분']
    
    #지도에 데이터 찍어서 보여주기
    folium.Marker([sub_lat,sub_long],tooltip = title).add_to(m)

#한글이 안나오는 오류로 html로 trouble shooting 
m.save('example.html')
m

#### 찍히는 마커 모양을 다르게 변형할 수도 있습니다.
동그라미 모양으로 데이터를 표시하고 싶을 때는 CircleMarker을 사용합니다.

In [ ]:
#지도 새로 띄우기
m = folium.Map([lat,long],zoom_start=9)

for i in ex.index:
    sub_lat =  ex.loc[i,'위도']
    sub_long = ex.loc[i,'경도']
    
    title = ex.loc[i,'구분']
  
    #구분이 소매면 빨간색으로 표시, default는 녹색
    
    color = 'green'
    if ex.loc[i,'구분'] == '소매':
        color = "red"
        
    #지도에 동그라미로 데이터 찍기    
    folium.CircleMarker([sub_lat,sub_long],color=color,radius = 5, tooltip=title).add_to(m)

#한글이 안나오는 오류로 html로 trouble shooting
m.save('example.html')
m

## [Python] Folium – 지리정보 시각화

In [ ]:
import folium

m = folium.Map(
    location=[37.5838699,127.0565831],
    zoom_start=15
)

m.save('map.html')

Marker
Marker 클래스를 이용하면 특정 위치를 아이콘으로 강조할 수 있다.

Icon
기본적으로 Glyphicons에서 제공하는 이미지를 이용해 지도 위에 표시할 수 있다.

In [ ]:
import folium

m = folium.Map(
    location=[37.5838699,127.0565831],
    zoom_start=15
)

folium.Marker(
  location=[37.5838699,127.0565831],
  popup='University of Seoul',
  icon=folium.Icon(color='red',icon='star')
).add_to(m)

m.save('map.html')

In [ ]:
import folium

m = folium.Map(
    location=[37.5838699,127.0565831],
    zoom_start=15
)

folium.CircleMarker(
  [37.5838699,127.0565831],
  radius=100,
  color='#ffffgg',
  fill_color='#fffggg',
  popup='University of Seoul'
).add_to(m)

m.save('map.html')

In [ ]:
import folium
import pandas as pd
import urllib.request
import datetime
import time
import json
import webbrowser

#[CODE 1]
def get_request_url(url):
    req = urllib.request.Request(url)
    req.add_header("X-Naver-Client-Id", "NAVER API ID")
    req.add_header("X-Naver-Client-Secret", "NAVER API SECRET")
    try:
        response = urllib.request.urlopen(req)
        if response.getcode() == 200:
            print ("[%s] Url Request Success" % datetime.datetime.now())
            return response.read().decode('utf-8')
    except Exception as e:
        print(e)
        print("[%s] Error for URL : %s" % (datetime.datetime.now(), url))
        return None

#[CODE 2]
def getGeoData(address):
    base = "https://openapi.naver.com/v1/map/geocode"
    
    try:
        parameters = "?query=%s" % urllib.parse.quote(address)
    except:
        return None
    
    url = base + parameters
    
    retData = get_request_url(url)
    if retData == None:
        return None

    jsonAddress = json.loads(retData)

    if 'result' in jsonAddress.keys():
        latitude = jsonAddress['result']['items'][0]['point']['y']
        longitude = jsonAddress['result']['items'][0]['point']['x']
    else:
        return None
    
    return [latitude, longitude]

def main():
   
    #[CODE 3]
    map = folium.Map(location=[37.5103, 126.982], zoom_start=12)

    filename = './dataset/서울시 초등학교 현황.csv'
    #df = pd.DataFrame.from_csv(filename, encoding='CP949', index_col=0, header=0)
    df = pd.read_csv(filename, encoding='CP949', index_col=0, header=0)
    geoData = []
    
    #[CODE 4]
    for index, row in df.iterrows():
        geoData = getGeoData(row['주소'])
        if geoData != None:
            folium.Marker(geoData, popup=row['학교명'], icon=folium.Icon(color='red')).add_to(map)
    
    svFilename = './dataset/elementary_school.html'
    map.save(svFilename)
    webbrowser.open(svFilename)  
    
if __name__ == "__main__":
    main()

In [ ]:
import pandas as pd
import folium
import webbrowser
 
# Load the shape of the zone (US states)
# Find the original file here: https://github.com/python-visualization/folium/tree/master/examples/data
# You have to download this file and set the directory where you saved it
state_geo = 'TL_SCCO_SIG_WGS84.json'
 
# Load the unemployment value of each state
# Find the original file here: https://github.com/python-visualization/folium/tree/master/examples/data
state_unemployment = 'Total_People_2018.csv'
state_data = pd.read_csv(state_unemployment, encoding = 'euc-kr')
 
# Initialize the map:
m = folium.Map(location=[36, 127], tiles="OpenStreetMap", zoom_start=7)
 
# Add the color for the chloropleth:

m.choropleth(
 geo_data=state_geo,
 name='choropleth',
 data=state_data,
 columns=['Code', 'Population'],
 key_on='feature.properties.SIG_CD',
 fill_color='YlGn',
 fill_opacity=0.7,
 line_opacity=0.5,
 legend_name='Population Rate (%)'
)

folium.LayerControl().add_to(m)
 
# Save to html
m.save('folium_kr.html')
webbrowser.open_new("folium_kr.html")

In [ ]:
m = folium.Map(
    location=[37.55, 126.98],
    zoom_start=8
)
m

#folium.Map.save() 를 통해서 html 파일로 저장 가능

## Marker

In [ ]:
m = folium.Map(
    location=[37.55, 126.98],
    zoom_start=8
)

folium.Marker(
    location=[37.55, 126.98],
    icon = folium.Icon() # Icon Marker 지정
).add_to(m)
m

 
icon에 색상이나 star와 같은 모양을 지정하는 것 또한 가능합니다.
 
https://getbootstrap.com/docs/3.3/components/#glyphicons-glyph
 
위 사이트에 접속하면 marker에 사용할 수 있는 icon을 확인할 수 있습니다.
In [6]:

In [ ]:
m = folium.Map(
    location=[37.55, 126.98],
    zoom_start=8
)

folium.Marker(
    location=[37.55, 126.98],
    icon = folium.Icon(color='red', icon='star') # Icon Marker 지정
).add_to(m)
m

## popup

popup 속성을 통해 문자열을 지정하여 아이콘을 클릭했을 때 팝업의 문구를 지정할 수 있습니다.
In [7]:

In [ ]:
m = folium.Map(
    location=[37.55, 126.98],
    zoom_start=8
)

folium.Marker(
    location=[37.55, 126.98],
    popup = 'Seoul',
    icon = folium.Icon(color='red', icon='star') # Icon Marker 지정
).add_to(m)
m

## Circle Marker

CircleMarker를 통해 특정 위치를 원형의 마커로 표시할 수 있습니다.
In [8]:

In [ ]:
m = folium.Map(
    location=[37.55, 126.98],
    zoom_start=12
)

folium.Marker(
    location=[37.55, 126.98],
    popup = 'Seoul',
    icon = folium.Icon(color='red', icon='star') # Icon Marker 지정
).add_to(m)

# CircleMarker
folium.CircleMarker(
    location=[37.58091849226215, 126.92331631718088],
    popup='Myongji Univ.',
    radius=20, # CircleMarker 크기 지정
    color='blue', # CirclaMarker 테두리 색상
    fill_color='#000000' # CircleMarker 내부 색상
).add_to(m)
m

## Marker Cluster

MarkerCluster는 가까운 거리의 Marker를 하나의 군집으로 Clustering 하여 시각화하는 방법입니다. Map의 Zoom에 따라 군집의 갯수가 달라지는 interactive한 시각화 방법으로 사용됩니다.
In [10]:


In [ ]:
ls -l

In [ ]:
import pandas as pd

df = pd.read_csv('./dataset/소상공인시장진흥공단_상가(상권)정보_서울_20200630.csv',sep='|')
starbucks = df[df['상호명'].str.contains('스타벅스|starbucks|STARBUCKS', na=False)]

In [ ]:
m = folium.Map(
    location=[37.55, 126.98],
    zoom_start=10
)

latlong = starbucks[['위도', '경도']]

from folium.plugins import MarkerCluster
marker_cluster = MarkerCluster().add_to(m)

for lat, long in zip(latlong.위도, latlong.경도):
    folium.Marker([lat,long], icon = folium.Icon(color="green")).add_to(marker_cluster)

m

## GeoJson

GeoJson을 이용하면 다각형을 표현할 수 있는데 이것을 통해 행정구역과 같은 경계선을 표현하는데 효과적입니다.
In [12]:

In [ ]:
m = folium.Map(
    location=[37.55, 126.98],
    zoom_start=10
)

latlong = starbucks[['위도', '경도']]

from folium.plugins import MarkerCluster
marker_cluster = MarkerCluster().add_to(m)

for lat, long in zip(latlong.위도, latlong.경도):
    folium.Marker([lat,long], icon = folium.Icon(color="green")).add_to(marker_cluster)

# GeoJson 코드
import json
with open('./dataset/seoul_municipalities_geo.json',mode='rt',encoding='utf-8') as f:
    geo = json.loads(f.read())
    f.close()
    
folium.GeoJson(
    geo,
    name='seoul_municipalities'
).add_to(m)

m

In [ ]:
ls -l ./dataset/

## 1. 엑셀 데이터 불러오기 및 특정 컬럼 추출

In [ ]:
pip install pyproj

In [ ]:
pip install geopandas

# [python, folium] 서울 따릉이 대여소 지도 시각화

## 1. 엑셀 데이터 불러오기 및 특정 컬럼 추출

#### 따릉이 대여소 좌표 데이트는 여기서 받은 : data.seoul.go.kr/dataList/OA-13252/F/1/datasetView.do 

In [ ]:
from pyproj import Transformer
import pyproj
import numpy as np
import pandas  as pd
from shapely.geometry import Point as point
import geopandas as gpd
import folium


bike = pd.read_csv('./dataset/공공자전거 대여소 정보(21.01.31 기준).csv',encoding='cp949') # 해당 엑셀 파일 불러오기

In [ ]:
bike.head()

In [ ]:
type(bike)

In [ ]:
bike = bike.rename({'대여소\n번호':'대여소ID','Unnamed: 3':'상세주소', 'Unnamed: 4':'위도', 'Unnamed: 5':'경도'}, axis='columns')

In [ ]:
bike

In [ ]:
bike = bike.dropna(subset=['대여소ID','위도', '경도'])

In [ ]:
bike

In [ ]:
bike = bike.drop(index=1, axis=0)
bike.head()

In [ ]:
bike.reset_index(inplace=True)

In [ ]:
bike.head()

In [ ]:
bike.drop('index', axis=1, inplace=True)
bike

In [ ]:
bike.info()

In [ ]:
bike['경도'] = bike['경도'].astype(float)
bike['위도'] = bike['위도'].astype(float)

In [ ]:
bike

In [ ]:
bike['위도'].isnull().sum()

In [ ]:
#bike = bike.dropna(subset=['위도', '경도'])

In [ ]:
bike_id = bike['대여소ID']
bike_x = bike['위도']
bike_y = bike['경도']

In [ ]:
bike_x.isnull().sum()

In [ ]:
bike_y.isnull().sum()

In [ ]:
bike

## 2. 죄표계 변환 후.shp로 보내기

In [ ]:
# Case 1

'''
- x, y 좌표와 대여소ID를 받아서 dict 형태로 만들고 
- x, y 좌표를 QGIS 지도 좌표계에 맞게 변환 (to EPSG:3857)
- 후에 point들을 .shp로 내보내기 
'''

TRAN_4326_TO_3857 = Transformer.from_crs("EPSG:4326", "EPSG:3857") # EPSG:3857좌표계로 변환
	# "ESPG:3857"을 다른 좌표계로 바꾸면 무한하게 응용 가능

points = []
ttareung = []
print(len(bike))
for i in range(len(bike)-1): # 마지막 인덱스에 NaN이 존재해서 -1

    idx = bike_id[i]
    x_coord = bike_x[i]
    y_coord = bike_y[i]
    
    xx, yy = TRAN_4326_TO_3857.transform(x_coord, y_coord) # 변환하는 과정
    bike_dict = {'ID': idx, 'x_coord': xx, 'y_coord': yy}
    
    if i % 20 == 0: 
        print(f'{i}th station is working...')
    
    points.append(point(xx, yy))
    ttareung.append(bike_dict)

points = gpd.GeoDataFrame(geometry = points)
points.to_file('./dataset/tta_points.shp', driver="Shapefile")

## 3. Folium을 활용한 지도 시각화

In [ ]:
bike

In [ ]:
bike.isnull().sum()


In [ ]:
# Case 2
'''
- x, y 좌표를 받아서 좌표계 변환 없이 m.html에 시각화하기 
'''
#지도의 중심을 지정하기 위해 위도와 경도의 평균 구하기
lat = bike['위도'].mean()
long = bike['경도'].mean()

#지도 띄우기
m = folium.Map([lat, long], zoom_start = 9)

coords = []
for i in range(len(bike)-1):
    x = bike_x[i]
    y = bike_y[i]
    coords.append([x, y])
print(len(coords))
for i in range(len(coords)):
    folium.Circle(
        location = coords[i],
        radius = 50,
        color = '#000000',
        fill = 'crimson',
    ).add_to(m)

m.save('./dataset/tta_points.html')

In [ ]:
m

In [ ]:
pwd

In [ ]:
ls -l

## [Python] folium 을 이용하여 서울시 동별 인구 수 시각화하기

#### 1. 사용할 패키지 들을 import 합니다.

In [ ]:
import json
import folium
import pandas as pd

#### 2. 지도를 그릴때 사용할 json 파일을 불러 옵니다.

'''
이 json 파일은 https://github.com/southkorea/southkorea-maps 이곳에 있는
https://raw.githubusercontent.com/southkorea/southkorea-maps/master/kostat/2013/json/skorea_submunicipalities_geo_simple.json
이 json 파일을 조금 수정하여 사용하였습니다.

위의 json 파일은 전국의 모든 동의 위치 정보를 담고 있습니다.
따라서, 서울에 존재하는 동의 위치정보만 따로 가져와서 사용하였습니다.
'''

In [ ]:
geo_path = './dataset/skorea_submunicipalities_geo_simple.json'
geo_str = json.load(open(geo_path, encoding='utf-8'))
#geo_str

## 3. 지도에 표시할 서울시 동별 인구데이터를 불러옵니다.

#### 서울시 동별 인구데이터는 아래와같이 dong, counts라는 컬럼을 갖고 있으며,
dong은 서울시 행정동코드를 의미합니다.
counts는 인구수를 의미합니다. ( 실제 인구수와 다를 수 있습니다. 그냥 하나의 sample데이터라고 생각해 주세요. )
아래와 같이 엑셀파일에 저장되어 있습니다. 

In [ ]:
dongData = pd.read_csv('./dataset/2020년_서울시_동별_인구.csv', encoding='cp949')
dongData.head()

In [ ]:
dongData['2020년_총인구수'].head()

In [ ]:
type(dongData)

In [ ]:
dongData = dongData.rename({'행정구역':'dong', '2020년_총인구수':'counts'}, axis='columns')
dongData.head()

In [ ]:
dongData = dongData.drop(index=0, axis=0)
dongData.head()

In [ ]:
dongData['dong'] = dongData['dong'].str[-10:-1] #.split(' ')
dongData.head()

In [ ]:
dongData.reset_index(inplace=True)
dongData.head()

In [ ]:
dongData.head()

In [ ]:
dongData.drop('index', axis=1, inplace=True)
dongData.head()

In [ ]:
for i in range(len(dongData)):
    dongData.loc[i, "dong"] = str(dongData.loc[i, "dong"])
dongData.head()

## 4. folium 패키지를 이용하여 Map을 생성합니다.

In [ ]:
dongData.isnull().sum()

In [ ]:
dongData.info()

In [ ]:
dongData.head()

In [ ]:
dongData['counts'] = dongData['counts'].str.replace(',','')
dongData['counts'] = dongData['counts'].astype(int)
dongData.head()

In [ ]:
#dongData['dong'] = dongData['dong'].astype(int)
dongData.info()

In [ ]:
seoul_map = folium.Map(location=[37.5502, 126.982], zoom_start=10.5, tiles='cartodbpositron')
seoul_map.choropleth(geo_data=geo_str,
                     data=dongData,
                     columns=['dong', 'counts'],
                     fill_color='YlGn', #'PuRd',
                     key_on='properties.code',
                     highlight=True,
                     fill_opacity=0.5, line_opacity=1,
                     legend_name='population(people)')

## 5. 생성한 map을 html 파일로 저장하고, 화면에 출력합니다.

In [ ]:
seoul_map.save('seoul_map.html')
seoul_map

## folium 으로 지도 시작화하기 with python

#### 시군구별 총수입금액평균을 folium으로 시각화 해 보았습니다.

#### fill_color = 'BuGn', 'BuPu', 'GnBu', 'OrRd', 'PuBu', 'PuBuGn', 'PuRd', 'RdPu', 'YlGn', 'YlGnBu', 'YlOrBr', 'YlOrRd'

In [ ]:
import webbrowser
import folium, json
import pandas as pd

m = folium.Map(location=[36, 127], tiles="OpenStreetMap", zoom_start=8)
geo_data = json.load(open('./dataset/skorea_municipalities_geo_simple.json', encoding='utf-8'))

df = pd.read_csv('./dataset/ddd.csv', encoding='utf-8', dtype={'code':'str'})
folium.Choropleth(geo_data=geo_data,
                  data = df,
                  columns=['sigun', 'avg_income'],
                  key_on='feature.properties.name',
                  fill_color='YlGn', #OrRd, PuBu, PuBuGn, GnBu, BuPu, BuGn, PuBuGn, PuRd, RdPu, YlGn, YlGnBu, YlOrBr, YlOrRd
                  fill_opacity=0.8
                  ).add_to(m)

m.save('folium_kr.html')
webbrowser.open_new("folium_kr.html")

In [ ]:
m

In [ ]:
pwd